In [150]:
from selenium import webdriver
import os
from SICOIN_scraper import *
import xlrd
import re

In [2]:
options = webdriver.ChromeOptions()
options.headless = False # True
# options.add_argument("download.default_directory="+os.getcwd())
options.add_experimental_option("prefs", {
  "download.default_directory": os.path.abspath("../../../Resource Tracking/SICOIN GT/SICOIN - Gestion"),
  "download.prompt_for_download": False,
})
driver = webdriver.Chrome(chrome_options=options)
driver.get('https://sicoin.minfin.gob.gt')

/home/guillermo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [8]:
login(driver)
#driver.close()

In [70]:
# IGRC01 - Ejecución Financiera
# TB
for year in range(2012, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "17"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh

In [ ]:
# IGRC03 - Comparacion de Ejecucion Financiera y Fisica
# TB
for year in range(2012, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "17"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

In [69]:
# IGRC02 - Ejecución Física
# TB
for year in range(2012, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "17"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

In [71]:
# IGRC00, IGRC01, IGRC02
# Malaria
for year in range(2013, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "18"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "18"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "18"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

In [ ]:
# IGRC00, IGRC01, IGRC02
# HIV
for year in range(2013, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "16"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "16"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "16"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

# Preprocessing:

In [177]:
cols_igrc1 = [
    "Program", "Subprograma", "Proyecto", "Actividad-Obra", "Renglon" 
    "ASIGNADO", "MODIFICADO", "VIGENTE", "DEVENGADO", "PorcentajeEJEC"
    ]

In [ ]:
def nextCell(row, offset = 0):
    lenr = len(row)
    if offset >= lenr:
        return None
    while (offset+1 < lenr) & (row[offset].ctype == xlrd.XL_CELL_EMPTY):
        offset += 1
    return offset

def SICOIN_extractTable(sheet, start, cols):
    print("Extractng ", sheet.nrows, "rows")
    year = sheet.cell_value(rowx = 14, colx = 10)
    i = start
    data = []
    continuousEmpty = 0
    while True:
        i += 1
        if i >= sheet.nrows:
            break
        row = sheet.row(i)
        j = 0
        j = nextCell(row, j)
        j0 = j
        if j <= 9:
            continuousEmpty = 0

            code = str(row[j].value).strip()
            if code == "TOTAL":
                continue

            j = nextCell(row, j+1)
            name = str(row[j].value).strip()
            
            cod_index = mapCodes[j0]
            codes[cod_index] = name
            codes["cod_" + cod_index] = code

            j = nextCell(row, j+1)
            if j is None:
                continue
            if j == 63: 
                continue

            rowData = list(codes.values())
            while True:
                value = str(row[j].value).strip()
                rowData.append(value)
                j = nextCell(row, j+1)
                if j is None:
                    break

            data.append(rowData)
        elif j == 63:
            # empty row
            continuousEmpty += 1
        if continuousEmpty > 20:
            print("Breaking because exceeded cont empty rows limit. Extracted until row", i)
            break;
    df = pd.DataFrame(data = data, columns = cols)
    df["Year"] = year
    return df

In [13]:
# Read all excel files:
xls_files = []
for root, dirs, files in os.walk("../../../Resource Tracking/SICOIN GT/SICOIN - Gestion"):
    n = len(files)
    for file in files:
        print(file)
        xls_files.append(xlrd.open_workbook(root + "/" + file))

_172.25.3.18reportes$00820869rkxmi5zwc0agaf0veighmspr123040.xls
_172.25.3.18reportes$00820967rkxmi5zwc0agaf0veighmspr122627.xls
_172.25.3.18reportes$00820869rkxmi5zwc0agaf0veighmspr122725.xls
_172.25.3.18reportes$00820869rkxmi5zwc0agaf0veighmspr120707.xls
_172.25.3.18reportes$00820967rkxmi5zwc0agaf0veighmspr112726.xls
_172.25.3.18reportes$00820850rkxmi5zwc0agaf0veighmspr114340.xls
_172.25.3.18reportes$00820869rkxmi5zwc0agaf0veighmspr115714.xls
_172.25.3.18reportes$00820850rkxmi5zwc0agaf0veighmspr121642.xls
_172.25.3.18reportes$00820850rkxmi5zwc0agaf0veighmspr115919.xls
_172.25.3.18reportes$00820967rkxmi5zwc0agaf0veighmspr120557.xls
_172.25.3.18reportes$00820850rkxmi5zwc0agaf0veighmspr114301.xls
_172.25.3.18reportes$00820967rkxmi5zwc0agaf0veighmspr120934.xls
_172.25.3.18reportes$00820869rkxmi5zwc0agaf0veighmspr121727.xls
_172.25.3.18reportes$00820967rkxmi5zwc0agaf0veighmspr122934.xls
_172.25.3.18reportes$00820869rkxmi5zwc0agaf0veighmspr122030.xls
_172.25.3.18reportes$00820850rkxmi5zwc0a

In [176]:
for x in xls_files:
    s = x.sheet_by_index(0)
    year =  0
    program = None
    dataset = None
    if str(s.row(3)[2].value).endswith("IGRC01"):
        year = s.row(15)[0].value.strip()[-4:]
        dataset = "EjecucionFin"
        program = s.row(13)[2].value
    elif str(s.row(1)[5].value).endswith("IGRC02"):
        year = s.row(15)[8].value
        dataset = "EjecucionFisica"
        program = s.row(8)[5].value
    elif str(s.row(1)[6].value).endswith("IGRC03"):
        year = s.row(15)[10].value
        dataset = "EjecucionFinFis"
        program = s.row(8)[6].value
    else:
        print("Bad report")
    if program:
        program_pre = re.search("entidad (\w* )?\= (\d*)\, programa \= (\d*)", 
                                                program, re.IGNORECASE)
        if program_pre is not None:
            program = "-".join(program_pre.groups()[1:3])
    print(dataset, year, program)

EjecucionFinFis 2018 11130009-16
EjecucionFisica 2017 11130009-16
EjecucionFinFis 2017 11130009-16
EjecucionFinFis 2017 11130009-18
EjecucionFisica 2015 11130009-17
EjecucionFin 2014 11130009-17
EjecucionFinFis 2014 11130009-18
EjecucionFin 2014 11130009-16
EjecucionFin 2015 11130009-18
EjecucionFisica 2017 11130009-18
EjecucionFin 2012 11130009-17
EjecucionFisica 2018 11130009-18
EjecucionFinFis 2014 11130009-16
EjecucionFisica 2018 11130009-16
EjecucionFinFis 2015 11130009-16
EjecucionFin 2016 11130009-18
EjecucionFisica 2018 11130009-17
EjecucionFinFis 2018 11130009-18
EjecucionFin 2016 11130009-17
EjecucionFinFis 2017 11130009-17
EjecucionFisica 2017 11130009-17
EjecucionFin 2015 11130009-17
EjecucionFinFis 2013 11130009-16
EjecucionFin 2017 11130009-16
EjecucionFisica 2014 11130009-18
EjecucionFinFis 2016 11130009-18
EjecucionFin 2016 11130009-16
EjecucionFisica 2014 11130009-16
EjecucionFisica 2013 11130009-18
EjecucionFinFis 2013 11130009-18
EjecucionFisica 2015 11130009-18
Ejec

In [156]:
x = xls_files[5]

In [157]:
s = x.sheet_by_index(0)

In [171]:
s.row(13)[2].value, re.search("entidad (\w* )?\= (\d*)\, programa \= (\d*)", s.row(13)[2].value, re.IGNORECASE).groups()

('Entidad Institucional = 11130009, Programa = 17',
 ('Institucional ', '11130009', '17'))